In [ ]:
import json
import datetime
from telegram import Update
from telegram.ext import (
    Application, CommandHandler, MessageHandler, filters, ContextTypes
)
from pytz import timezone

# ===== CONFIG =====
BOT_TOKEN = "803418297:AAGLOyj0IciXM14hWit0qjmNA-Tu-P2oMw"
CHAT_ID = "2098144295"
TASK_FILE = "tasks.json"
LOG_FILE = "logs.txt"
LOCAL_TZ = timezone("Asia/Dhaka")  # for timestamps

# ===== Helper Functions =====
def load_tasks():
    try:
        with open(TASK_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except:
        return []

def save_tasks(tasks):
    with open(TASK_FILE, "w", encoding="utf-8") as f:
        json.dump(tasks, f)

def write_log(message: str):
    now = datetime.datetime.now(LOCAL_TZ).strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"{now} - {message}\n")
    return f"{now} - {message}"

# ===== Command Handlers =====
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "👋 Hi! I'm your ML Study Assistant Bot.\n\n"
        "📌 To-Do Commands:\n"
        "   /add <task> → Add a task\n"
        "   /list → Show tasks\n"
        "   /done <num> → Mark task as done\n"
        "   /clear → Clear all tasks\n\n"
        "📝 Logging Commands:\n"
        "   /log <message> → Save daily progress\n"
        "   /showlogs → Show last 5 logs\n"
        "   /clearlogs → Clear all logs\n\n"
        "💡 Or just send me a text — I will log it automatically!"
    )

# ----- To-Do List -----
async def add(update: Update, context: ContextTypes.DEFAULT_TYPE):
    task = " ".join(context.args)
    if not task:
        await update.message.reply_text("⚠️ Usage: /add <task>")
        return
    tasks = load_tasks()
    tasks.append({"task": task, "done": False})
    save_tasks(tasks)
    await update.message.reply_text(f"✅ Task added: {task}")

async def list_tasks(update: Update, context: ContextTypes.DEFAULT_TYPE):
    tasks = load_tasks()
    if not tasks:
        await update.message.reply_text("📭 No tasks yet!")
        return
    msg = "📋 Your Tasks:\n"
    for i, t in enumerate(tasks, 1):
        status = "✅" if t["done"] else "❌"
        msg += f"{i}. {t['task']} {status}\n"
    await update.message.reply_text(msg)

async def done(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        index = int(context.args[0]) - 1
        tasks = load_tasks()
        if index < 0 or index >= len(tasks):
            raise IndexError
        tasks[index]["done"] = True
        save_tasks(tasks)
        await update.message.reply_text(f"🎉 Task {index+1} marked as done!")
    except:
        await update.message.reply_text("⚠️ Invalid input! Usage: /done <task_number>")

async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE):
    save_tasks([])
    await update.message.reply_text("🗑️ All tasks cleared!")

# ----- Logs -----
async def log(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if not context.args:
        await update.message.reply_text("⚠️ Usage: /log <message>")
        return
    message = " ".join(context.args)
    log_entry = write_log(message)
    await update.message.reply_text(f"📝 Log saved:\n{log_entry}")

async def show_logs(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        with open(LOG_FILE, "r", encoding="utf-8") as f:
            lines = f.readlines()
        if not lines:
            await update.message.reply_text("📂 No logs yet!")
            return
        last_logs = "".join(lines[-5:])
        await update.message.reply_text(f"📑 Last 5 Logs:\n\n{last_logs}")
    except FileNotFoundError:
        await update.message.reply_text("⚠️ No logs file found.")

async def clear_logs(update: Update, context: ContextTypes.DEFAULT_TYPE):
    open(LOG_FILE, "w", encoding="utf-8").close()
    await update.message.reply_text("🗑️ All logs cleared!")

# ----- Default Handler (auto-log any plain text) -----
async def save_task(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = update.message.text.strip()
    if not text.startswith("/"):  # ignore commands
        log_entry = write_log(text)
        await update.message.reply_text(f"✅ Logged:\n{log_entry}")
    else:
        await update.message.reply_text("⚠️ Unknown command. Type /start to see available commands.")

# ===== Main =====
def main():
    app = Application.builder().token(BOT_TOKEN).build()

    # Command handlers
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("add", add))
    app.add_handler(CommandHandler("list", list_tasks))
    app.add_handler(CommandHandler("done", done))
    app.add_handler(CommandHandler("clear", clear))
    app.add_handler(CommandHandler("log", log))
    app.add_handler(CommandHandler("showlogs", show_logs))
    app.add_handler(CommandHandler("clearlogs", clear_logs))

    # Default text handler
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, save_task))

    app.run_polling()

if __name__ == "__main__":
    main()
